# Operation Barbarossa
(Imaginative scenario)

On 22 June 1941, Germany began a major attack on the Soviet Union, the communist state that consisted of Russia and a number of neighbouring countries. During the first nine months of the advance, one million German soldiers were killed and many we wounded. 

<img src="operation-barbarossa.png" alt="Drawing" style="width: 650px;"/>

The German government is loosing a lot of money and needs the cheapest way of transporting its backup units for the occupation of Leningrad, Moscow and Stalingrad. For the transition points of the units there are two cities where the soldiers can regroup in Waschau and Budapest.

OUT: Berlin, Leipzig, Munich

Transition: Warschau, Budapest

IN: Leningrad, Moscow, Stalingrad

# Mathematical definition
- c- Costs
- m- Arcs
- x- Flows
- k- Commodities
- n- Nodes
- Demand
- Origin
- Destination
### Objective function:
$$ minC = \sum_{k\epsilon K}^{} \sum_{m\epsilon L}^{} c_{m}^{k} \times x_{m}^{k} $$

### Set of Constraints:
$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = Dem^{k} \rightarrow i \epsilon O(k) $$

$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = -Dem^{k} \rightarrow i \epsilon D(k) $$

$$ \sum_{m\epsilon L_{i}^{out}}x_{m}^{k} - \sum_{m\epsilon L_{i}^{in}} x_{m}^{k} = 0 \rightarrow i \epsilon N, k \epsilon K $$

### Capacity:
$$ \sum_{k \epsilon K} x_{m}^{k} \leq  Cap_{m} \rightarrow m \epsilon L $$

$$ x_{m}^{k} \epsilon \mathbb{Z}_{0}^{+} $$





In [6]:
from pyomo.environ import *
import numpy as pd
import pandas as pd 
import numpy as np 

In [12]:
# Load file
List_arcs = pd.read_excel('dt2.xlsx', sheet_name = "Arcs", index_col=False)
List_commo = pd.read_excel('dt2.xlsx', sheet_name = "Commodities", index_col=False)
print(List_arcs)
print(List_commo)

    From  To  Cost  Capac
0      1   4   1.0     10
1      1   5   2.0     20
2      2   4   1.5     30
3      2   5   1.0     40
4      3   4   2.5     50
5      3   5   1.0     60
6      4   6   3.0    100
7      4   7   4.0    130
8      4   8   5.0    110
9      5   6   4.5     90
10     5   7   3.5    150
11     5   8   4.0    100
   From  To  Quant
0     1   7    100
1     1   8     40
2     2   6     80
3     2   7     90
4     3   8     70
5     3   6    120


In [27]:
# Define a model
model = ConcreteModel()

model.r = RangeSet(1,8) # 8 Nodes
print(len(model.r))
model.j = RangeSet(1,len(List_arcs['From'])) # 12 Arcs
print(len(model.j))
# Define decision variables
model.flow = Var(model.r, model.j,domain=NonNegativeReals)
print(len(model.flow))

8
12
96


In [28]:
# Define objective function
def obj_expression(model):
    return sum(model.flow[row['From'], row['To']] * row['Cost'] for _, row in List_arcs.iterrows())
model.OBJ = Objective(rule=obj_expression, sense=minimize)

In [11]:
# Define constraints
def flow_balance(model, node):
    inbound_flow = sum(model.flow[row['From'], node] for _, row in List_arcs.iterrows() if row['To'] == node)
    outbound_flow = sum(model.flow[node, row['To']] for _, row in List_arcs.iterrows() if row['From'] == node)
    return inbound_flow == outbound_flow
    
model.FlowBalance = Constraint(List_arcs['To'].unique(), rule=flow_balance)


def arc_capacity(model, from_node, to_node):
    flow = model.flow[from_node, to_node]
    row = List_arcs[(List_arcs['From'] == from_node) & (List_arcs['To'] == to_node)]
    return flow <= row['Capac'].iloc[0]
model.ArcCapacity = Constraint(List_arcs['From'], List_arcs['To'], rule=arc_capacity)

RuntimeError: Cannot add component 'FlowBalance_index' (type <class 'pyomo.core.base.set.FiniteScalarSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.FiniteScalarSet'>) is already defined.

In [ ]:
# Solve the optimization problem
solver = SolverFactory('gurobi')
results = solver.solve(model)
    
# Print the results
print("Objective value: ", model.OBJ())
for _, row in List_arcs.iterrows():
    print("Flow from {} to {}: {}".format(row['From'], row['To'], model.flow[row['From'], row['To']]))
